In [1]:
!pip install google-api-python-client

In [2]:
pip install langdetect

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pytube

In [4]:
pip install --upgrade google-auth-oauthlib google-auth-httplib2

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install emoji==1.7

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
from googleapiclient.discovery import build
import numpy as np
from pytube import Playlist,extract
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import statistics
from langdetect import detect
import emoji
import re
import text2emotion as te
 import json

[nltk_data] Downloading package stopwords to C:\Users\Ivan
[nltk_data]     Tai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Ivan
[nltk_data]     Tai\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Ivan
[nltk_data]     Tai\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
#https://www.youtube.com/watch?v=SwSbnmqk3zY

In [8]:
df_rommance_song=pd.read_csv("rommance_song_ytvideo_statistics.csv")

In [ ]:
secrets_filename = 'secret_keys'
with open(secrets_filename, 'r') as f:
    api_keys = json.loads(f.read())

In [9]:
youtube=build("youtube","v3",developerKey=api_key)

In [10]:
sia = SentimentIntensityAnalyzer()

In [11]:
text_list=[]
id_list=[]
pos_list=[]
neg_list=[]
neu_list=[]
compound_list=[]     

for id in df_rommance_song["id"]:   
    try:
        results = youtube.commentThreads().list(
        part="snippet",
        videoId=id,
        textFormat="plainText",
        maxResults=100
        ).execute()
    except:
        pass
    #Get the 100 comments from video
    for item in results["items"]:
        comment = item["snippet"]["topLevelComment"]
        text = comment["snippet"]["textDisplay"]
    #preprossessing of comment text
        try: # detect the language of comment 
          language=detect(text)
        except:
           language=" "
        #  filter the useful text
        if language=="en" and len(text)<300 :   
           text_list.append(text)
           id_list.append(id)
           sentiment_dict=sia.polarity_scores(text)
           pos_list.append(sentiment_dict["pos"])
           neg_list.append(sentiment_dict["neg"])
           neu_list.append(sentiment_dict["neu"])
           compound_list.append(sentiment_dict["compound"])



lists=[text_list,id_list,neg_list,neu_list,pos_list,
compound_list]
for list in lists:
    list=np.array(list)

      

In [12]:
comments_dict={"id":id_list,"text":text_list,"pos":pos_list,"neu":neu_list,
               "neg":neg_list,"overall_sentiment":compound_list}

In [13]:
df_rommance_comment=pd.DataFrame(comments_dict)
df_rommance_comment

,id,text,pos,neu,neg,overall_sentiment
0,Ip6cw8gfHHI,Who made it alive to 2025 ❤,0.342,0.658,0.000,0.3818
1,Ip6cw8gfHHI,I just knew that you are black. I mean how wou...,0.000,1.000,0.000,0.0000
2,Ip6cw8gfHHI,if you want comeback. come with me forever,0.157,0.843,0.000,0.0772
3,Ip6cw8gfHHI,Wow😮165M who,0.000,1.000,0.000,0.0000
4,Ip6cw8gfHHI,who's also here bcs of yoonchaee,0.000,1.000,0.000,0.0000
...,...,...,...,...,...,...
5122,OwP6U0LRzQM,So many memories when I listened to this song ...,0.000,1.000,0.000,0.0000
5123,OwP6U0LRzQM,I miss so bad this era of pop music🥹,0.000,0.527,0.473,-0.6590
5124,OwP6U0LRzQM,anyone here in 2026🎉🎉🎉🎉,0.000,1.000,0.000,0.0000
5125,OwP6U0LRzQM,"turn around baby still listening 🎧 July 9, 202...",0.000,1.000,0.000,0.0000


In [14]:

def emotion_dict1(df):
    emotion_dict=te.get_emotion(df)
    emotion_score1=emotion_dict["Angry"]
    return emotion_score1

def emotion_dict2(df):
    emotion_dict=te.get_emotion(df)
    emotion_score2=emotion_dict["Surprise"]
    return emotion_score2

def emotion_dict3(df):
    emotion_dict=te.get_emotion(df)
    emotion_score3=emotion_dict["Happy"]
    return emotion_score3

def emotion_dict4(df):
    emotion_dict=te.get_emotion(df)
    emotion_score4=emotion_dict["Fear"]
    return emotion_score4

def emotion_dict5(df):
    emotion_dict=te.get_emotion(df)
    emotion_score5=emotion_dict["Sad"]
    return emotion_score5

def extract_emotion(df):
    df["Angry"]=df["text"].apply(emotion_dict1)
    df["Surprise"]=df["text"].apply(emotion_dict2)
    df["Happy"]=df["text"].apply(emotion_dict3)
    df["Fear"]=df["text"].apply(emotion_dict4)
    df["Sad"]=df["text"].apply(emotion_dict5)
    

extract_emotion(df=df_rommance_comment)



In [49]:
def sentiment_category(score):
    if score>=0.5:  # Positive sentiment score
        return"Strongly Positive"  # High rating and positive sentiment
    elif 0.0 <= score < 0.5:
        return "Mildly Positive"
    elif -0.5 <= score < 0.0:  # Negative sentiment score
        return 'Mildly Negative'  # Low rating and negative sentiment
    else:
        return 'Strongly_Negative'  # Neutral rating and neutral sentiment

# Define a function to bucket sentiment scores into text ranges

def sentiment_range(score):    
    if score >= 0.5:
        return '0.5 to 1.0'  # Strongly positive sentiment
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49'  # Mildly positive sentiment
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0'  # Mildly negative sentiment
    else:
        return '-1.0 to -0.5'  # Strongly negative sentiment

In [50]:
#Apply sentiment categorization using both text and rating
df_rommance_comment["SentimentCategory"] = df_rommance_comment["overall_sentiment"].apply(sentiment_category)

# Apply sentiment bucketing to categorize scores into defined ranges
df_rommance_comment['SentimentBucket'] = df_rommance_comment["overall_sentiment"].apply(sentiment_range)

df_rommance_comment

,id,text,pos,neu,neg,overall_sentiment,SentimentCategory,SentimentBucket,Angry,Surprise,Happy,Fear,Sad
0,Ip6cw8gfHHI,Who made it alive to 2025 ❤,0.342,0.658,0.000,0.3818,Mildly Positive,0.0 to 0.49,0.0,0.0,0.0,0.0,1.0
1,Ip6cw8gfHHI,I just knew that you are black. I mean how wou...,0.000,1.000,0.000,0.0000,Mildly Positive,0.0 to 0.49,0.0,0.0,0.0,0.0,0.0
2,Ip6cw8gfHHI,if you want comeback. come with me forever,0.157,0.843,0.000,0.0772,Mildly Positive,0.0 to 0.49,0.0,0.0,0.0,0.0,0.0
3,Ip6cw8gfHHI,Wow😮165M who,0.000,1.000,0.000,0.0000,Mildly Positive,0.0 to 0.49,0.0,0.0,0.0,0.0,0.0
4,Ip6cw8gfHHI,who's also here bcs of yoonchaee,0.000,1.000,0.000,0.0000,Mildly Positive,0.0 to 0.49,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5122,OwP6U0LRzQM,So many memories when I listened to this song ...,0.000,1.000,0.000,0.0000,Mildly Positive,0.0 to 0.49,0.0,0.0,0.0,0.0,0.0
5123,OwP6U0LRzQM,I miss so bad this era of pop music🥹,0.000,0.527,0.473,-0.6590,Strongly_Negative,-1.0 to -0.5,0.0,0.0,0.0,0.0,1.0
5124,OwP6U0LRzQM,anyone here in 2026🎉🎉🎉🎉,0.000,1.000,0.000,0.0000,Mildly Positive,0.0 to 0.49,0.0,0.0,0.0,0.0,0.0
5125,OwP6U0LRzQM,"turn around baby still listening 🎧 July 9, 202...",0.000,1.000,0.000,0.0000,Mildly Positive,0.0 to 0.49,0.0,1.0,0.0,0.0,0.0


In [51]:
def emotion_category(score):
    if score>=0.5:  # Positive intense emotion score
        return"intense"  # intense emotion
    elif 0.0 < score < 0.5:
        return "Mild" #mild emotion 
    else:
        return "None"
# Define a function to bucket emotion scores into text ranges

def emotion_range(score):    
    if score >= 0.5:
        return '0.5 to 1.0'  # intense emotion
    elif 0.0 < score < 0.5:
        return '0.01 to 0.49'  # Mild emotion
    else: 
        return "0.00 to 0.00"

In [52]:
emotion_list=["Angry","Surprise","Happy","Fear","Sad"]

for emotion in emotion_list:
   df_rommance_comment[f"{emotion} category"]=df_rommance_comment[emotion].apply(emotion_category)
   df_rommance_comment[f"{emotion} Buckek"]=df_rommance_comment[emotion].apply(emotion_range)

df_rommance_comment

,id,text,pos,neu,neg,overall_sentiment,SentimentCategory,SentimentBucket,Angry,Surprise,...,Angry category,Angry Buckek,Surprise category,Surprise Buckek,Happy category,Happy Buckek,Fear category,Fear Buckek,Sad category,Sad Buckek
0,Ip6cw8gfHHI,Who made it alive to 2025 ❤,0.342,0.658,0.000,0.3818,Mildly Positive,0.0 to 0.49,0.0,0.0,...,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,intense,0.5 to 1.0
1,Ip6cw8gfHHI,I just knew that you are black. I mean how wou...,0.000,1.000,0.000,0.0000,Mildly Positive,0.0 to 0.49,0.0,0.0,...,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00
2,Ip6cw8gfHHI,if you want comeback. come with me forever,0.157,0.843,0.000,0.0772,Mildly Positive,0.0 to 0.49,0.0,0.0,...,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00
3,Ip6cw8gfHHI,Wow😮165M who,0.000,1.000,0.000,0.0000,Mildly Positive,0.0 to 0.49,0.0,0.0,...,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00
4,Ip6cw8gfHHI,who's also here bcs of yoonchaee,0.000,1.000,0.000,0.0000,Mildly Positive,0.0 to 0.49,0.0,0.0,...,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5122,OwP6U0LRzQM,So many memories when I listened to this song ...,0.000,1.000,0.000,0.0000,Mildly Positive,0.0 to 0.49,0.0,0.0,...,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00
5123,OwP6U0LRzQM,I miss so bad this era of pop music🥹,0.000,0.527,0.473,-0.6590,Strongly_Negative,-1.0 to -0.5,0.0,0.0,...,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,intense,0.5 to 1.0
5124,OwP6U0LRzQM,anyone here in 2026🎉🎉🎉🎉,0.000,1.000,0.000,0.0000,Mildly Positive,0.0 to 0.49,0.0,0.0,...,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00
5125,OwP6U0LRzQM,"turn around baby still listening 🎧 July 9, 202...",0.000,1.000,0.000,0.0000,Mildly Positive,0.0 to 0.49,0.0,1.0,...,None,0.00 to 0.00,intense,0.5 to 1.0,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00


In [53]:
df_rommance_comment[df_rommance_comment["overall_sentiment"].isnull()]

,id,text,pos,neu,neg,overall_sentiment,SentimentCategory,SentimentBucket,Angry,Surprise,...,Angry category,Angry Buckek,Surprise category,Surprise Buckek,Happy category,Happy Buckek,Fear category,Fear Buckek,Sad category,Sad Buckek


In [54]:
# group id with sentiment score
column_list=["neg","neu","pos","overall_sentiment","Sad","Angry","Surprise","Happy","Fear"]
df_comment_avg_score= df_rommance_comment.groupby(["id"])[column_list].mean().reset_index()

# add the new column to df_rock_song
for column in column_list:
    df_rommance_song[f"Avg_{column}"]=df_comment_avg_score[f"{column}"]


    df_rommance_song


In [55]:
# Apply sentiment categorization using both text and rating
df_rommance_song["SentimentCategory"] = df_rommance_song["Avg_overall_sentiment"].apply(sentiment_category)

# Apply sentiment bucketing to categorize scores into defined ranges
df_rommance_song['SentimentBucket'] = df_rommance_song["Avg_overall_sentiment"].apply(sentiment_range)

# Apply emotion bucketing to categorize scores into defined ranges
# Apply emotion categorization using both text and rating


for emotion in emotion_list:
   df_rommance_song[f"{emotion} category"]=df_rommance_song[f"Avg_{emotion}"].apply(emotion_category)
   df_rommance_song[f"{emotion} Buckek"]=df_rommance_song[f"Avg_{emotion}"].apply(emotion_range)

df_rommance_song

,title,id,channel_title,date,viewCount,likeCount,commentCount,musician,song_title,Year,...,Angry category,Angry Buckek,Surprise category,Surprise Buckek,Happy category,Happy Buckek,Fear category,Fear Buckek,Sad category,Sad Buckek
0,d4vd - Here With Me [Official Music Video],Ip6cw8gfHHI,d4vdVEVO,2022-11-23 20:00:09,165316192,1656184,27566,d4vd,Here With Me,2022,...,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49
1,"Lady Gaga, Bruno Mars - Die With A Smile (Offi...",kPa7bsKwL-c,LadyGagaVEVO,2024-08-16 04:00:36,515910345,6775697,227956,"Lady Gaga, Bruno Mars",Die With A Smile,2024,...,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49
2,Damiano David - Born With a Broken Heart (Offi...,Z4-g8UXa944,DamianoDavidVEVO,2024-10-25 11:00:06,11213241,220566,7888,Damiano David,Born With a Broken Heart,2024,...,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49
3,Billie Eilish - BIRDS OF A FEATHER (BILLIE BY ...,1IUCyniCWzg,BillieEilishVEVO,2024-05-22 22:54:49,42950141,534432,9010,Billie Eilish,BIRDS OF A FEATHER,2024,...,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49
4,"Dax - ""From A Man's Perspective"" (Official Mus...",NvBfUq8pOkA,Dax,2024-12-27 17:00:30,1656735,92484,4797,Dax,"""From A Man's Perspective""",2024,...,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,James Bay - Peer Pressure ft. Julia Michaels,e-9aaLDIb_Y,JamesBayVEVO,2019-03-29 15:58:14,14388469,134755,1450,James Bay,Peer Pressure ft. Julia Michaels,2019,...,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49
96,Jason Derulo - Marry Me [Official HD Music Video],zRCsZ5a3aCM,Jason Derulo,2013-09-23 14:29:03,191774082,1140307,35347,Jason Derulo,Marry Me,2013,...,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49
97,Little Mix - Secret Love Song (Official Video)...,qgy7vEje5-w,littlemixVEVO,2016-02-03 08:00:00,553042471,3511639,98791,Little Mix,Secret Love Song,2016,...,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49
98,Simple Plan - Summer Paradise ft. Sean Paul (O...,qjHlgrGsLWQ,SimplePlan,2012-03-31 12:42:00,146300738,849442,54500,Simple Plan,Summer Paradise ft. Sean Paul,2012,...,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49


In [56]:
df_rommance_song=df_rommance_song.dropna(axis=0).reset_index(drop=True)
# drop the null value 

In [57]:
df_rommance_song

,title,id,channel_title,date,viewCount,likeCount,commentCount,musician,song_title,Year,...,Angry category,Angry Buckek,Surprise category,Surprise Buckek,Happy category,Happy Buckek,Fear category,Fear Buckek,Sad category,Sad Buckek
0,d4vd - Here With Me [Official Music Video],Ip6cw8gfHHI,d4vdVEVO,2022-11-23 20:00:09,165316192,1656184,27566,d4vd,Here With Me,2022,...,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49
1,"Lady Gaga, Bruno Mars - Die With A Smile (Offi...",kPa7bsKwL-c,LadyGagaVEVO,2024-08-16 04:00:36,515910345,6775697,227956,"Lady Gaga, Bruno Mars",Die With A Smile,2024,...,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49
2,Damiano David - Born With a Broken Heart (Offi...,Z4-g8UXa944,DamianoDavidVEVO,2024-10-25 11:00:06,11213241,220566,7888,Damiano David,Born With a Broken Heart,2024,...,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49
3,Billie Eilish - BIRDS OF A FEATHER (BILLIE BY ...,1IUCyniCWzg,BillieEilishVEVO,2024-05-22 22:54:49,42950141,534432,9010,Billie Eilish,BIRDS OF A FEATHER,2024,...,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49
4,"Dax - ""From A Man's Perspective"" (Official Mus...",NvBfUq8pOkA,Dax,2024-12-27 17:00:30,1656735,92484,4797,Dax,"""From A Man's Perspective""",2024,...,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,James Bay - Peer Pressure ft. Julia Michaels,e-9aaLDIb_Y,JamesBayVEVO,2019-03-29 15:58:14,14388469,134755,1450,James Bay,Peer Pressure ft. Julia Michaels,2019,...,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49
96,Jason Derulo - Marry Me [Official HD Music Video],zRCsZ5a3aCM,Jason Derulo,2013-09-23 14:29:03,191774082,1140307,35347,Jason Derulo,Marry Me,2013,...,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49
97,Little Mix - Secret Love Song (Official Video)...,qgy7vEje5-w,littlemixVEVO,2016-02-03 08:00:00,553042471,3511639,98791,Little Mix,Secret Love Song,2016,...,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49
98,Simple Plan - Summer Paradise ft. Sean Paul (O...,qjHlgrGsLWQ,SimplePlan,2012-03-31 12:42:00,146300738,849442,54500,Simple Plan,Summer Paradise ft. Sean Paul,2012,...,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49,Mild,0.01 to 0.49


In [58]:
# switch the column

In [61]:
df_rommance_comment.to_csv("rommance_song_ytcomment.csv",index=False)
df_rommance_song.to_csv("rommance_song_ytvideo_statistics.csv", index=False)



In [60]:
df_rommance_comment

,id,text,pos,neu,neg,overall_sentiment,SentimentCategory,SentimentBucket,Angry,Surprise,...,Angry category,Angry Buckek,Surprise category,Surprise Buckek,Happy category,Happy Buckek,Fear category,Fear Buckek,Sad category,Sad Buckek
0,Ip6cw8gfHHI,Who made it alive to 2025 ❤,0.342,0.658,0.000,0.3818,Mildly Positive,0.0 to 0.49,0.0,0.0,...,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,intense,0.5 to 1.0
1,Ip6cw8gfHHI,I just knew that you are black. I mean how wou...,0.000,1.000,0.000,0.0000,Mildly Positive,0.0 to 0.49,0.0,0.0,...,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00
2,Ip6cw8gfHHI,if you want comeback. come with me forever,0.157,0.843,0.000,0.0772,Mildly Positive,0.0 to 0.49,0.0,0.0,...,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00
3,Ip6cw8gfHHI,Wow😮165M who,0.000,1.000,0.000,0.0000,Mildly Positive,0.0 to 0.49,0.0,0.0,...,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00
4,Ip6cw8gfHHI,who's also here bcs of yoonchaee,0.000,1.000,0.000,0.0000,Mildly Positive,0.0 to 0.49,0.0,0.0,...,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5122,OwP6U0LRzQM,So many memories when I listened to this song ...,0.000,1.000,0.000,0.0000,Mildly Positive,0.0 to 0.49,0.0,0.0,...,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00
5123,OwP6U0LRzQM,I miss so bad this era of pop music🥹,0.000,0.527,0.473,-0.6590,Strongly_Negative,-1.0 to -0.5,0.0,0.0,...,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,intense,0.5 to 1.0
5124,OwP6U0LRzQM,anyone here in 2026🎉🎉🎉🎉,0.000,1.000,0.000,0.0000,Mildly Positive,0.0 to 0.49,0.0,0.0,...,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00
5125,OwP6U0LRzQM,"turn around baby still listening 🎧 July 9, 202...",0.000,1.000,0.000,0.0000,Mildly Positive,0.0 to 0.49,0.0,1.0,...,None,0.00 to 0.00,intense,0.5 to 1.0,None,0.00 to 0.00,None,0.00 to 0.00,None,0.00 to 0.00
